# Quickstart (pytorch)

## Working with data

In [2]:
import torch
from torch import nn # also import torch.nn
# from torch.utils.data import Dataset
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

In [3]:
#downloading training data from open datasets
training_data = datasets.FashionMNIST(
    root='data',
    train= True,
    download=True,
    transform=ToTensor(),
)

#downloading test data from open datasets
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [4]:
training_data[0]

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0039, 0.0000, 0.0000, 0.0510,
           0.2863, 0.0000, 0.0000, 0.0039, 

In [5]:
batch_size = 64 #no of sample data iterable each cycle 

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Creating Models

In [6]:
# choosing device
device = (
    "cuda" if torch.cuda.is_available() else "cpu"
)
print(f'Using {device} device ')

Using cpu device 


In [7]:
#defining the model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__() # super() gives access to the super class in this case is nn.Module, this whole line is basically calling the initializer of the superclass i.e nn.Module
        self.flatten = nn.Flatten() # Flatten as its name is used to convert 28*28 images to 784 dimensional vector
        # relu-> rectified linear unit is an activation function that brings non-linearity to the models.
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self,x):
        x=self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Optimizing the model parameters

In [8]:
#we need loss function and optimizers
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)

In [9]:
def train(dataloader,model,loss_fn,optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X,y = X.to(device) , y.to(device)

        #computing prediction error
        pred = model(X)
        loss = loss_fn(pred,y)

        #backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0 :
            loss,current = loss.item(), (batch+1)*len(X)
            print(f"loss:{loss:>7f} [{current:>5d}/{size:>5d}]")


In [ ]:
# also checking the model performance against the test dataset to ensure it is learning.
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
# epoch is one complete pass through the entire dataset by the model.
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss:2.291727 [   64/60000]
loss:2.283017 [ 6464/60000]
loss:2.267150 [12864/60000]
loss:2.259468 [19264/60000]
loss:2.245358 [25664/60000]
loss:2.218641 [32064/60000]
loss:2.219482 [38464/60000]
loss:2.188143 [44864/60000]
loss:2.180873 [51264/60000]
loss:2.139854 [57664/60000]
Test Error: 
 Accuracy: 45.2%, Avg loss: 2.138962 

Epoch 2
-------------------------------
loss:2.151261 [   64/60000]
loss:2.136190 [ 6464/60000]
loss:2.079785 [12864/60000]
loss:2.083939 [19264/60000]
loss:2.038942 [25664/60000]
loss:1.982821 [32064/60000]
loss:1.994860 [38464/60000]
loss:1.918749 [44864/60000]
loss:1.921480 [51264/60000]
loss:1.834491 [57664/60000]
Test Error: 
 Accuracy: 53.9%, Avg loss: 1.841750 

Epoch 3
-------------------------------
loss:1.882788 [   64/60000]
loss:1.845480 [ 6464/60000]
loss:1.730096 [12864/60000]
loss:1.759205 [19264/60000]
loss:1.670427 [25664/60000]
loss:1.621083 [32064/60000]
loss:1.630707 [38464/60000]
loss:1.540952 [44864